In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import configparser
import time
import tweepy

In [2]:
path_conf = 'conf/settings.ini'
config = configparser.ConfigParser()
config.read(path_conf)
keys = config['KEYS']

In [3]:
TWITTER_APP_KEY = keys['TWITTER_APP_KEY']
TWITTER_APP_SECRET = keys['TWITTER_APP_SECRET']
TWITTER_KEY = keys['TWITTER_KEY']
TWITTER_SECRET = keys['TWITTER_SECRET']

auth = tweepy.OAuthHandler(TWITTER_APP_KEY, TWITTER_APP_SECRET)
auth.set_access_token(TWITTER_KEY, TWITTER_SECRET)

api = tweepy.API(auth)

In [4]:
from streamz import Stream
from streamz.dataframe import DataFrame
from datetime import datetime as dt
import pandas as pd
import json
from IPython.display import clear_output

# build listener for new tweets

# create a listener that prints the text of any tweet that comes from the Twitter API.
class StreamListener(tweepy.StreamListener):
  def __init__(self, time_limit=60):
    # set time limit and counter
    self.start_time = time.time()
    self.limit = time_limit
    self.counter = 0
    # define source and sink
    self.source = Stream()
    self.sink = {}
    # count hashtags
    clear_output()
    self.source\
      .map(self.hashtags)\
      .accumulate(self.sum_hashtags)\
      .sink(self.sink.update)
    super(StreamListener, self).__init__()
  
  def on_status(self, status):
    # skip retweets
    if hasattr(status, "retweeted_status"):
      return
    # process tweets
    if hasattr(status, "extended_tweet"):
      text = status.extended_tweet["full_text"]
    else:
      text = status.text
    id_str = status.id_str
    name = status.user.screen_name
    ts = dt.now().timestamp()
    # create data dict
    data = {'id': id_str, 'name': name, 'text': text, 'ts': ts}
    # emit data to source
    self.source.emit(data)
    self.counter += 1
    # check for sink cleanup
    if self.counter % 1000 == 0:
      self.clean_sink()
    # check for end
    if (time.time() - self.start_time) < self.limit:
      return True
    else:
      return False

  def on_error(self, status_code):
    print(f'Error: {status_code}')
    if status_code == 420:
      print('Too many requests!')
  
  def hashtags(self, data):
    words = data['text'].split()
    hashtags = list(filter(lambda s: s.startswith('#'), words))
    hashtags = dict((hashtag, 1) for hashtag in hashtags)
    return(hashtags)
  
  def sum_hashtags(self, x, y):
    for key in y:
      if key in x:
        x[key] += 1
      else:
        x[key] = 1
    return(x)
  
  def get_counter(self):
    return(self.counter)

  def get_hashtags(self):
    return(self.sink)
  
  def clean_sink(self):
    for key in self.sink:
      value = self.sink[key]
      if value == 1:
        self.sink.pop(key)
  
  def get_filter_hashtags(self):
    out = {}
    for key in self.sink:
      value = self.sink[key]
      if value > 1:
        out[key] = value
    return(out)
  
  def get_filter_hashtags_df(self):
    htags = self.get_filter_hashtags()
    keys = list(htags.keys())
    values = list(htags.values())
    out = pd.DataFrame({'hashtags': keys, 'count': values})
    out = out.sort_values('count', ascending=False)
    return(out)

In [5]:
stream_listener = StreamListener(time_limit=3600)
tracker = ['trump', 'donald trump']

In [6]:
stream_listener.running = False

In [7]:
stream = tweepy.Stream(auth=api.auth, listener=stream_listener)
stream.filter(track=tracker, is_async=True)

In [78]:
stream_listener.get_filter_hashtags_df()

,hashtags,count
10,#Trump,26
2,#COVID19,9
0,#trump,7
8,#MAGA,5
11,#US,5
1,#USA,4
12,#Covid,4
20,#Biden2020,4
50,#Hamilton,4
51,#MASHUP,4


## Test section

In [199]:
with open('twitter.json', 'r') as f:
  dump = json.load(f)
  for data in dump:
    if data['id'] == '1282929875997749249':
      sample = data
def hashtags(data):
    words = data['text'].split()
    hashtags = list(filter(lambda s: s.startswith('#'), words))
    hashtags = dict((hashtag, 1) for hashtag in hashtags)
    return(hashtags)
test = hashtags(sample)
print(test)

{'#trump2020': 1, '#MakeChinaPay': 1, '#Vote2020': 1, '#voteoutdemocrats': 1}


In [201]:
x = {'#trump2020': 1, '#MakeChinaPay': 1, '#Vote2020': 1, '#voteoutdemocrats': 1}
y = {'#Trump': 1, '#Biden,': 1}
z = {'#Trump': 1, '#Corona,': 1}
def sum_hashtags(x, y):
  for key in y:
    if key in x:
      x[key] += 1
    else:
      x[key] = 1
  return(x)
test = sum_hashtags(x, y)
test = sum_hashtags(x, z)
print(test)

{'#trump2020': 1, '#MakeChinaPay': 1, '#Vote2020': 1, '#voteoutdemocrats': 1, '#Trump': 2, '#Biden,': 1, '#Corona,': 1}


In [213]:
from streamz import Stream

tsample = '@realDonaldTrump Let China pay for this\n\nVote Trump this November \n\n#trump2020 #MakeChinaPay #Vote2020 #voteoutdemocrats'

source = Stream()
source\
  .map(hashtags)\
  .accumulate(sum_hashtags)\
  .sink(print)
#source.map(hashtags).sink(print)

Output()

In [215]:
source.emit(sample)

{'#trump2020': 2, '#MakeChinaPay': 2, '#Vote2020': 2, '#voteoutdemocrats': 2}


```
description = status.user.description
loc = status.user.location
text = status.text
coords = status.coordinates
name = status.user.screen_name
user_created = status.user.created_at
followers = status.user.followers_count
id_str = status.id_str
created = status.created_at
retweets = status.retweet_count
bg_color = status.user.profile_background_color
```

interesting objects attributes
```
text: short text with link
user: object(id, name, screen_name, location)
extended_tweet(full_text, user_mentions)
```